In [1]:
import pandas as pd
import pyodbc
import warnings
import numpy as np
from datetime import datetime

warnings.simplefilter('ignore')

In [2]:
FROM_DB_PATH = r"..\\..\\..\\Data\\Raw\\Sqlite\\AdventureWorks.mdb"

FROM_DB_CONN_PATH = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + FROM_DB_PATH

TO_DB_CONN_PATH = r"DRIVER={SQL Server};SERVER=143.177.235.181,1433;DATABASE=AdventureWorks;UID=SA;PWD=Str0ngP@ssword"

FROM_DB = pyodbc.connect(FROM_DB_CONN_PATH)
import_cursor = FROM_DB.cursor()

TO_DB = pyodbc.connect(TO_DB_CONN_PATH)
export_cursor = TO_DB.cursor()

In [3]:
tablesBACK = ["""Person_Address", "Production_BillOfMaterials","Production_ProductCategory",
          "Production_Product","Purchasing_PurchaseOrderHeader","Purchasing_PurchaseOrderDetail","Purchasing_Vendor","Sales_SalesTerritory",
          "Sales_Store","Sales_Customer","Sales_SalesOrderHeader"""]

tables = ["Sales_SalesOrderHeader"]

In [ ]:
for table in tables:
    try:
        import_cursor.execute(f"SELECT * FROM {table}")
        columns = [column[0] for column in import_cursor.description]

        rows = import_cursor.fetchall()
        if not rows:
            print(f"Geen data voor {table}")
            continue

        placeholders = ", ".join(["?" for _ in columns])
        # columns = [f"[{col}]" if col.lower() == "group" else col for col in columns]  VOOR SALES_TERRITORY
        insert_query = f"INSERT INTO {table} ({', '.join(columns)}) VALUES ({placeholders})"

        export_cursor.fast_executemany = True

        fout = 0

        for row in rows:
            try:
                export_cursor.execute(insert_query, row)
            except Exception as row_error:
                print(f"Fout bij invoegen van een rij in {table}: {row_error}")
                fout+=1

        TO_DB.commit()
        print(f"Tabel {table} voltooid. with errors: {fout}")

    except Exception as e:
        print(f"Fout bij tabel {table} + {e}")


Fout bij invoegen van een rij in Sales_SalesOrderHeader: ('23000', '[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "FK__Sales_Sal__BillT__6477ECF3". The conflict occurred in database "AdventureWorks", table "dbo.Person_Address", column \'AddressID\'. (547) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)')
Fout bij invoegen van een rij in Sales_SalesOrderHeader: ('23000', '[23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "FK__Sales_Sal__BillT__6477ECF3". The conflict occurred in database "AdventureWorks", table "dbo.Person_Address", column \'AddressID\'. (547) (SQLExecDirectW); [23000] [Microsoft][ODBC SQL Server Driver][SQL Server]The statement has been terminated. (3621)')
Fout bij invoegen van een rij in Sales_SalesOrderHeader: ('23000', '[23000] [Microsoft][ODBC SQL Server Driv